In [1]:
import cv2
import os

In [2]:
print(os.listdir(os.path.join(os.getcwd(),"videos")))


['JDwduqfuHjc.mp4', 'video1_1080p.mp4', 'video1_360p.mp4', 'video1_720p.mp4']


In [3]:
path = os.path.join(os.getcwd(),"videos")

In [4]:
import ffmpeg


ModuleNotFoundError: No module named 'ffmpeg'

In [5]:
input_file = path+'/JDwduqfuHjc.mp4'
output_file = 'output_file.ts'


In [6]:
import av
import av.datasets


In [22]:
input_container = av.open(input_file)
output_container = av.open(output_file, 'w')

# Select the video stream and transcode it
for packet in input_container.demux():
    for frame in packet.decode():
        if isinstance(frame, av.video.frame.VideoFrame):
            frame = frame.reformat(format='yuv420p')
            output_container.mux(frame)

# Close the containers
input_container.close()
output_container.close()


TypeError: 'av.video.frame.VideoFrame' object is not iterable

In [24]:
import cv2
import os
import pandas as pd

# Define input parameters
video_links = [
   "https://www.youtube.com/watch?v=E_VKZ6dzauo"
]
output_path = os.getcwd()

# Initialize dataframe to store video information
video_df = pd.DataFrame(columns=["video_id", "video_url", "video_parts", "frame_rate"])

# Loop through each video link and extract information
for i, video_link in enumerate(video_links):
    # Extract video ID and URL from video link
    video_id = f"video{i+1}"
    video_url = video_link
    
    # Download video from YouTube using youtube-dl
    #os.system(f"youtube-dl -f mp4 {video_url} -o {video_id}.mp4")

    # Open video using OpenCV
    cap = cv2.VideoCapture(input_file)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_duration = num_frames / frame_rate

    # Split video into 10-second chunks
    chunk_length = 10
    chunk_start = 0
    chunk_end = chunk_start + chunk_length
    part_num = 1
    video_parts = []
    while chunk_start < video_duration:
        # Set output path for chunk
        output_chunk_path = os.path.join(output_path, f"{video_id}-p{part_num}.mp4")

        # Use OpenCV to extract chunk from video
        cap.set(cv2.CAP_PROP_POS_FRAMES, chunk_start * frame_rate)
        out = cv2.VideoWriter(output_chunk_path, cv2.VideoWriter_fourcc(*"mp4v"), frame_rate, (int(cap.get(3)), int(cap.get(4))))
        while cap.get(cv2.CAP_PROP_POS_FRAMES) < chunk_end * frame_rate:
            ret, frame = cap.read()
            if ret:
                out.write(frame)
            else:
                break
        out.release()

        # Add chunk information to video_parts list
        video_parts.append(f"{video_id}-p{part_num}.mp4")
        part_num += 1

        # Update chunk start and end times
        chunk_start = chunk_end
        chunk_end = chunk_start + chunk_length

    # Convert bitrate to 2 frames per second
    bitrate = int(num_frames / video_duration * 2)

    # Add video information to dataframe
    video_df = video_df.append({
        "video_id": video_id,
        "video_url": video_url,
        "video_parts": ", ".join(video_parts),
        "frame_rate": frame_rate,
        "bitrate": bitrate
    }, ignore_index=True)

    # Release video capture object
    cap.release()

# Print final dataframe
print(video_df)


  video_id                                    video_url  \
0   video1  https://www.youtube.com/watch?v=E_VKZ6dzauo   

                                         video_parts frame_rate  bitrate  
0  video1-p1.mp4, video1-p2.mp4, video1-p3.mp4, v...         23     46.0  


/tmp/ipykernel_4110/3208271125.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  video_df = video_df.append({


In [25]:
video_df

,video_id,video_url,video_parts,frame_rate,bitrate
0,video1,https://www.youtube.com/watch?v=E_VKZ6dzauo,"video1-p1.mp4, video1-p2.mp4, video1-p3.mp4, v...",23,46.0


In [7]:
import os
import pandas as pd
import cv2

# function to split video into 10-second chunks
def split_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    chunk_size = int(fps * 10)  # 10-second chunks
    chunks = [(i, min(i+chunk_size, total_frames)) for i in range(0, total_frames, chunk_size)]
    frame_indices = [int(i*fps)+1 for i in range(10)]  # indices of frames to keep in each chunk
    return chunks, frame_indices

# function to create dataframe with video data
def create_dataframe(video_urls):
    data = {'video_id': [], 'video_url': [], 'video_parts': [], 'frame_rate': []}
    for video_url in video_urls:
        video_id = video_url.split('=')[-1]
        video_path = f'{video_id}.mp4'
        #video_path = input_file
        #os.system(f'youtube-dl {video_url} -o "{video_path}"')  # download video
        chunks, frame_indices = split_video(video_path)
        for i, chunk in enumerate(chunks):
            start, end = chunk
            part_path = f'{video_id}-p{i+1}.mp4'
            os.system(f'ffmpeg -i {video_path} -vf select="eq(n\,{",".join(map(str, frame_indices))})" -c:v libx264 -c:a aac -strict experimental -b:a 128k {part_path}')
            data['video_id'].append(video_id)
            data['video_url'].append(video_url)
            data['video_parts'].append(part_path)
            data['frame_rate'].append(len(frame_indices))  # 2 frames per second
    df = pd.DataFrame(data)
    return df

# create dataframe
video_urls = ["https://www.youtube.com/watch?v=JDwduqfuHjc"]
df = create_dataframe(video_urls)

# save video chunks in current directory
for part_path in df['video_parts']:
    os.system(f'mv {part_path} {os.getcwd()}')

# print dataframe
print(df)


ffmpeg version 4.3.5-0+deb11u1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/aarch64-linux-gnu --incdir=/usr/include/aarch64-linux-gnu --arch=arm64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidsta

ffmpeg version 4.3.5-0+deb11u1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/aarch64-linux-gnu --incdir=/usr/include/aarch64-linux-gnu --arch=arm64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidsta

ffmpeg version 4.3.5-0+deb11u1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/aarch64-linux-gnu --incdir=/usr/include/aarch64-linux-gnu --arch=arm64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidsta

      video_id                                    video_url  \
0  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
1  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
2  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
3  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
4  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
5  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
6  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   

          video_parts  frame_rate  
0  JDwduqfuHjc-p1.mp4          10  
1  JDwduqfuHjc-p2.mp4          10  
2  JDwduqfuHjc-p3.mp4          10  
3  JDwduqfuHjc-p4.mp4          10  
4  JDwduqfuHjc-p5.mp4          10  
5  JDwduqfuHjc-p6.mp4          10  
6  JDwduqfuHjc-p7.mp4          10  


mv: 'JDwduqfuHjc-p7.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p7.mp4' are the same file


In [31]:
df

,video_id,video_url,video_parts,frame_rate
0,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p1.mp4,10
1,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p2.mp4,10
2,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p3.mp4,10
3,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p4.mp4,10
4,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p5.mp4,10
5,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p6.mp4,10
6,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p7.mp4,10


In [9]:
import os
import pandas as pd
import cv2

# function to split video into 10-second chunks
def split_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    chunk_size = int(fps * 10)  # 10-second chunks
    chunks = [(i, min(i+chunk_size, total_frames)) for i in range(0, total_frames, chunk_size)]
    frame_indices = [int(i*fps)+1 for i in range(10)]  # indices of frames to keep in each chunk
    return chunks, frame_indices

# function to create dataframe with video data
def create_dataframe(video_urls):
    data = {'video_id': [], 'video_url': [], 'video_parts': [], 'frame_rate': []}
    for video_url in video_urls:
        video_id = video_url.split('=')[-1]
        video_path = f'{video_id}.mp4'
        #os.system(f'youtube-dl {video_url} -o "{video_path}"')  # download video
        chunks, frame_indices = split_video(video_path)
        for i, chunk in enumerate(chunks):
            start, end = chunk
            part_path = f'{video_id}-p{i+1}.mp4'
            cap = cv2.VideoCapture(video_path)
            out = cv2.VideoWriter(part_path, cv2.VideoWriter_fourcc(*'mp4v'), 2, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
            
            for j in range(start, end):
                ret, frame = cap.read()
                if not ret:
                    break
                if j in frame_indices:
                    out.write(frame)
            out.release()
            data['video_id'].append(video_id)
            data['video_url'].append(video_url)
            data['video_parts'].append(part_path)
            data['frame_rate'].append(len(frame_indices))  # 2 frames per second
    df = pd.DataFrame(data)
    return df

# create dataframe
video_urls = ["https://www.youtube.com/watch?v=JDwduqfuHjc"]
df = create_dataframe(video_urls)

# save video chunks in current directory
for part_path in df['video_parts']:
    os.system(f'mv {part_path} {os.getcwd()}')

# print dataframe
print(df)


      video_id                                    video_url  \
0  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
1  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
2  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
3  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
4  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
5  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
6  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   

          video_parts  frame_rate  
0  JDwduqfuHjc-p1.mp4          10  
1  JDwduqfuHjc-p2.mp4          10  
2  JDwduqfuHjc-p3.mp4          10  
3  JDwduqfuHjc-p4.mp4          10  
4  JDwduqfuHjc-p5.mp4          10  
5  JDwduqfuHjc-p6.mp4          10  
6  JDwduqfuHjc-p7.mp4          10  


mv: 'JDwduqfuHjc-p1.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p1.mp4' are the same file
mv: 'JDwduqfuHjc-p2.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p2.mp4' are the same file
mv: 'JDwduqfuHjc-p3.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p3.mp4' are the same file
mv: 'JDwduqfuHjc-p4.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p4.mp4' are the same file
mv: 'JDwduqfuHjc-p5.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p5.mp4' are the same file
mv: 'JDwduqfuHjc-p6.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p6.mp4' are the same file
mv: 'JDwduqfuHjc-p7.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p7.mp4' are the same file


In [22]:
import base64

# function to convert image to byte string
def image_to_byte_string(image):
    _, buffer = cv2.imencode('.jpg', image)
    byte_string = base64.b64encode(buffer).decode()
    return byte_string

# add columns for each frame
for i in range(10):
    df[f'frame_{i+1}'] = ''

# iterate over each row and extract frames
for index, row in df.iterrows():
    video_parts = row['video_parts']
    frames = []
    for i in range(10):
        frame_path = f'{video_parts[:-4]}-frame{i+1}.jpg'
        #frame_path = f'/home/siddharthsk7t/DL/DL-temp/frames/frame{i+1}.jpg'

        frame = cv2.imread(frame_path)
        byte_string = image_to_byte_string(frame)
        frames.append(byte_string)
        os.remove(frame_path)
    df.loc[index, df.columns[4:]] = frames


[ WARN:0@1660.520] global loadsave.cpp:244 findDecoder imread_('/home/siddharthsk7t/DL/DL-temp/frames/frame1.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:1109: error: (-215:Assertion failed) !image.empty() in function 'imencode'


In [23]:
df

,video_id,video_url,video_parts,frame_rate,frame_1,frame_2,frame_3,frame_4,frame_5,frame_6,frame_7,frame_8,frame_9,frame_10
0,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p1.mp4,10,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIBAQ...
1,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p2.mp4,10,,,,,,,,,,
2,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p3.mp4,10,,,,,,,,,,
3,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p4.mp4,10,,,,,,,,,,
4,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p5.mp4,10,,,,,,,,,,
5,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p6.mp4,10,,,,,,,,,,
6,JDwduqfuHjc,https://www.youtube.com/watch?v=JDwduqfuHjc,JDwduqfuHjc-p7.mp4,10,,,,,,,,,,


In [21]:
import cv2
import os

# path to input video
video_path = 'JDwduqfuHjc-p1.mp4'

# create directory to store frames
if not os.path.exists('frames'):
    os.makedirs('frames')

# capture video
cap = cv2.VideoCapture(video_path)

# loop through frames and save as images
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if ret:
        # save frame as image
        cv2.imwrite(f'frames/frame{frame_count}.jpg', frame)
        frame_count += 1
    else:
        break

# release video capture and destroy window
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) /io/opencv/modules/highgui/src/window.cpp:1266: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [28]:
import os
import pandas as pd
import cv2

# function to split video into 10-second chunks
def split_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    chunk_size = int(fps * 10)  # 10-second chunks
    chunks = [(i, min(i+chunk_size, total_frames)) for i in range(0, total_frames, chunk_size)]
    frame_indices = [int(i*fps)+1 for i in range(10)]  # indices of frames to keep in each chunk
    return chunks, frame_indices

# function to create dataframe with video data
def create_dataframe(video_urls):
    data = {'video_id': [], 'video_url': [], 'video_parts': [], 'frame_rate': []}
    for video_url in video_urls:
        video_id = video_url.split('=')[-1]
        video_path = f'{video_id}.mp4'
        #os.system(f'youtube-dl {video_url} -o "{video_path}"')  # download video
        chunks, frame_indices = split_video(video_path)
        for i, chunk in enumerate(chunks):
            start, end = chunk
            part_path = f'{video_id}-p{i+1}.mp4'
            cap = cv2.VideoCapture(video_path)
            out = cv2.VideoWriter(part_path, cv2.VideoWriter_fourcc(*'mp4v'), 2, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
            
            frames = []
            for j in range(start, end):
                ret, frame = cap.read()
                if not ret:
                    break
                if j in frame_indices:
                    frames.append(frame)
                    out.write(frame)
            out.release()
            data['video_id'].append(video_id)
            data['video_url'].append(video_url)
            data['video_parts'].append(part_path)
            data['frame_rate'].append(len(frame_indices))  # 2 frames per second
            for f in range(10):
                data[f'frame_{f+1}'] = frames[f] if f < len(frames) else None
    df = pd.DataFrame(data)
    return df


# create dataframe
video_urls = ["https://www.youtube.com/watch?v=JDwduqfuHjc"]
df = create_dataframe(video_urls)

# save video chunks in current directory
for part_path in df['video_parts']:
    os.system(f'mv {part_path} {os.getcwd()}')

# print dataframe
print(df)


      video_id                                    video_url  \
0  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
1  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
2  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
3  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
4  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
5  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
6  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   

          video_parts  frame_rate frame_1 frame_2 frame_3 frame_4 frame_5  \
0  JDwduqfuHjc-p1.mp4          10    None    None    None    None    None   
1  JDwduqfuHjc-p2.mp4          10    None    None    None    None    None   
2  JDwduqfuHjc-p3.mp4          10    None    None    None    None    None   
3  JDwduqfuHjc-p4.mp4          10    None    None    None    None    None   
4  JDwduqfuHjc-p5.mp4          10    None    None    None    None    None   
5  JDwduqfuHjc-p6.mp4          10

mv: 'JDwduqfuHjc-p1.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p1.mp4' are the same file
mv: 'JDwduqfuHjc-p2.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p2.mp4' are the same file
mv: 'JDwduqfuHjc-p3.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p3.mp4' are the same file
mv: 'JDwduqfuHjc-p4.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p4.mp4' are the same file
mv: 'JDwduqfuHjc-p5.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p5.mp4' are the same file
mv: 'JDwduqfuHjc-p6.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p6.mp4' are the same file
mv: 'JDwduqfuHjc-p7.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p7.mp4' are the same file


In [ ]:
df

In [32]:
import os
import pandas as pd
import cv2
import io

# function to split video into 10-second chunks
def split_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    chunk_size = int(fps * 10)  # 10-second chunks
    chunks = [(i, min(i+chunk_size, total_frames)) for i in range(0, total_frames, chunk_size)]
    frame_indices = [int(i*fps)+1 for i in range(10)]  # indices of frames to keep in each chunk
    return chunks, frame_indices

def create_dataframe(video_urls):
    data = {'video_id': [], 'video_url': [], 'video_parts': [], 'frame_rate': []}
    for video_url in video_urls:
        video_id = video_url.split('=')[-1]
        video_path = f'{video_id}.mp4'
        #os.system(f'youtube-dl {video_url} -o "{video_path}"')  # download video
        chunks, frame_indices = split_video(video_path)
        for i, chunk in enumerate(chunks):
            start, end = chunk
            part_path = f'{video_id}-p{i+1}.mp4'
            cap = cv2.VideoCapture(video_path)
            out = cv2.VideoWriter(part_path, cv2.VideoWriter_fourcc(*'mp4v'), 2, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
            
            frames = []
            for j in range(start, end):
                ret, frame = cap.read()
                if not ret:
                    break
                if j in frame_indices:
                    frames.append(frame)
                    try:
                        cv2.imwrite(os.path.join(os.getcwd(),part_path), frame)
                    except Exception as e:
                        print(f"Error writing frame to file {part_path}: {e}")
                        
            out.release()
            data['video_id'].append(video_id)
            data['video_url'].append(video_url)
            data['video_parts'].append(part_path)
            data['frame_rate'].append(len(frame_indices))  # 2 frames per second
            for f in range(10):
                data[f'frame_{f+1}'] = frames[f] if f < len(frames) else None
    df = pd.DataFrame(data)
    return df


# create dataframe
video_urls = ["https://www.youtube.com/watch?v=JDwduqfuHjc"]
df = create_dataframe(video_urls)

# save video chunks in current directory
for part_path in df['video_parts']:
    os.system(f'mv {part_path} {os.getcwd()}')

# print dataframe
print(df)


Error writing frame to file JDwduqfuHjc-p1.mp4: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:692: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'

Error writing frame to file JDwduqfuHjc-p1.mp4: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:692: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'

Error writing frame to file JDwduqfuHjc-p1.mp4: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:692: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'

Error writing frame to file JDwduqfuHjc-p1.mp4: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:692: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'

Error writing frame to file JDwduqfuHjc-p1.mp4: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:692: error: (-2:

mv: 'JDwduqfuHjc-p1.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p1.mp4' are the same file
mv: 'JDwduqfuHjc-p2.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p2.mp4' are the same file
mv: 'JDwduqfuHjc-p3.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p3.mp4' are the same file
mv: 'JDwduqfuHjc-p4.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p4.mp4' are the same file
mv: 'JDwduqfuHjc-p5.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p5.mp4' are the same file
mv: 'JDwduqfuHjc-p6.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p6.mp4' are the same file
mv: 'JDwduqfuHjc-p7.mp4' and '/home/siddharthsk7t/DL/DL-temp/JDwduqfuHjc-p7.mp4' are the same file


In [ ]:
NameError

In [ ]:
import os
import pandas as pd
import cv2
import shutil
import base64

# function to split video into 10-second chunks
def split_video(video_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    chunk_size = int(fps * 10)  # 10-second chunks
    chunks = [(i, min(i+chunk_size, total_frames)) for i in range(0, total_frames, chunk_size)]
    frame_indices = [int(i*fps)+1 for i in range(10)]  # indices of frames to keep in each chunk
    return chunks, frame_indices

# function to convert image to byte string
def image_to_byte_string(image):
    _, buffer = cv2.imencode('.jpg', image)
    byte_string = base64.b64encode(buffer).decode()
    return byte_string

# function to create dataframe with video data
def create_dataframe(video_urls):
    data = {'video_id': [], 'video_url': [], 'video_parts': [], 'frame_rate': []}
    for video_url in video_urls:
        video_id = video_url.split('=')[-1]
        video_path = f'{video_id}.mp4'
        #os.system(f'youtube-dl {video_url} -o "{video_path}"')  # download video
        chunks, frame_indices = split_video(video_path)
        for i, chunk in enumerate(chunks):
            start, end = chunk
            part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
            os.makedirs(part_fpath, exist_ok = True)
            part_path = f'{os.path.join(part_fpath, f"{video_id}-p{i+1}.mp4")}'
            #print(f"DJ={part_path}")
            #print(f"DJ2={part_fpath}")
            video_chunk= f'{video_id}-p{i+1}.mp4'
            cap = cv2.VideoCapture(video_path)
            out = cv2.VideoWriter(part_path, cv2.VideoWriter_fourcc(*'mp4v'), 2, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
            
            for j in range(start, end):
                ret, frame = cap.read()
                if not ret:
                    break
                if j in frame_indices:
                    out.write(frame)
            out.release()

    






        
            data['video_id'].append(video_id)
            data['video_url'].append(video_url)
            data['video_parts'].append(video_chunk)
            data['frame_rate'].append(len(frame_indices))  # 2 frames per second
    df = pd.DataFrame(data)
    return df

# create dataframe
video_urls = ["https://www.youtube.com/watch?v=JDwduqfuHjc"]
df = create_dataframe(video_urls)

save_path = r"C:\Users\SkyLake\Documents\Python 2.0\Proj02b\HEXA"

# save video chunks in target directory
#for part_path in df['video_parts']:
    #shutil.copy2(part_path, save_path)
    #shutil.move(part_path, save_path)
    #print(f'moved {part_path} to {save_path}')

# print dataframe
print(df)


WOKRING V@IM


In [6]:
# function to convert video to image
def vid2frames(video_chunk_num):
    # path to input video
    part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
    part_frame_folder_path = re.search(r"(p\d+)", video_chunk_num).group()
    part_frame_path = os.path.join(part_fpath,part_frame_folder_path)
    main_video_path =f'{os.path.join(part_fpath, video_chunk_num)}'
    
    print(part_frame_path)
    os.makedirs(part_frame_path, exist_ok = True)
   


    video_path = main_video_path
    print(video_path)


    # capture video
    cap = cv2.VideoCapture(video_path)
    print(cap)
    # loop through frames and save as images
    frame_count = 1
    while cap.isOpened():
        ret, frame = cap.read()
        print("h1")
        if ret:
            
            # save frame as image
            cv2.imwrite(f'{part_frame_path}/{video_chunk_num}-frame{frame_count}.jpg', frame)
            print(f'{video_chunk_num}/frame{frame_count}.jpg')
            frame_count += 1
        else:
            break

    # release video capture and destroy window
    cap.release()
    cv2.destroyAllWindows()

In [7]:
vid2frames("JDwduqfuHjc-p1.mp4")

NameError: name 're' is not defined

In [8]:
CHUNK SOK 2 frams

SyntaxError: invalid syntax (3642543878.py, line 1)

In [9]:
import pandas as pd
import cv2
import os

# List of YouTube video URLs
video_urls = [
    "https://www.youtube.com/watch?v=JDwduqfuHjc"
]

# Create an empty list to store the data for each video
video_data = []

# Loop through each video URL
for url in video_urls:
    # Get the video ID from the URL
    video_id = url.split("=")[-1]
    # Download the video using youtube-dl
    os.system(f"youtube-dl -f 'bestvideo[ext=mp4]' -o 'video.mp4' {url}")
    # Open the video using OpenCV2
    cap = cv2.VideoCapture("JDwduqfuHjc.mp4")
    # Get the total number of frames
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    # Get the frame rate
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    # Split the video into 10-second chunks with two frames per second
    chunk_size = 10 * frame_rate
    for i in range(0, total_frames, chunk_size):
        # Set the start and end frames for the chunk
        start_frame = i
        end_frame = min(i + chunk_size, total_frames)
        # Get the chunk of frames
        frames = []
        for j in range(start_frame, end_frame):
            # Read the frame
            ret, frame = cap.read()
            # Keep only the first and middle frame
            if j == start_frame or j == (start_frame + end_frame) // 2:
                frames.append(frame)
        # Save the chunk as an mp4 file
        chunk_filename = f"{video_id}-p{int(i/chunk_size)+1}.mp4"

            # path to input video
        part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
        os.makedirs(part_fpath, exist_ok = True)
        main_video_path =f'{os.path.join(part_fpath, chunk_filename)}'


        cf = r"C:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts"
        frame_rate_chunk = 2
        out = cv2.VideoWriter(f"{main_video_path}", cv2.VideoWriter_fourcc(*'mp4v'), frame_rate_chunk, (frames[0].shape[1], frames[0].shape[0]))
        for frame in frames:
            out.write(frame)
        out.release()
        # Add the video data to the list
        video_data.append({
            "video_id": video_id,
            "video_url": url,
            "video_parts": chunk_filename,
            "frame_rate": frame_rate_chunk,
        })
    # Release the OpenCV2 capture object
    cap.release()

# Create the final data frame
df = pd.DataFrame(video_data)
print(df)


      video_id                                    video_url  \
0  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
1  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
2  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
3  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
4  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
5  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
6  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   

          video_parts  frame_rate  
0  JDwduqfuHjc-p1.mp4           2  
1  JDwduqfuHjc-p2.mp4           2  
2  JDwduqfuHjc-p3.mp4           2  
3  JDwduqfuHjc-p4.mp4           2  
4  JDwduqfuHjc-p5.mp4           2  
5  JDwduqfuHjc-p6.mp4           2  
6  JDwduqfuHjc-p7.mp4           2  


In [ ]:
import pandas as pd
import cv2
import os

# List of YouTube video URLs
video_urls = [
    "https://www.youtube.com/watch?v=JDwduqfuHjc"
]

# Create an empty list to store the data for each video
video_data = []

def chunker(video_urls_list):
    # Loop through each video URL
    for url in video_urls_list:
        # Get the video ID from the URL
        video_id = url.split("=")[-1]
        # Download the video using youtube-dl
        os.system(f"youtube-dl -f 'bestvideo[ext=mp4]' -o 'video.mp4' {url}")
        # Open the video using OpenCV2
        cap = cv2.VideoCapture("JDwduqfuHjc.mp4")
        # Get the total number of frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # Get the frame rate
        frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
        # Split the video into 10-second chunks with two frames per second
        chunk_size = 10 * frame_rate
        for i in range(0, total_frames, chunk_size):
            # Set the start and end frames for the chunk
            start_frame = i
            end_frame = min(i + chunk_size, total_frames)
            # Get the chunk of frames
            frames = []
            for j in range(start_frame, end_frame):
                # Read the frame
                ret, frame = cap.read()
                # Keep only the first and middle frame
                if j == start_frame or j == (start_frame + end_frame) // 2:
                    frames.append(frame)
            # Save the chunk as an mp4 file
            chunk_filename = f"{video_id}-p{int(i/chunk_size)+1}.mp4"

                # path to input video
            part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
            os.makedirs(part_fpath, exist_ok = True)
            main_video_path =f'{os.path.join(part_fpath, chunk_filename)}'


            cf = r"C:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts"
            frame_rate_chunk = 2
            out = cv2.VideoWriter(f"{main_video_path}", cv2.VideoWriter_fourcc(*'mp4v'), frame_rate_chunk, (frames[0].shape[1], frames[0].shape[0]))
            for frame in frames:
                out.write(frame)
            out.release()
            # Add the video data to the list
            video_data.append({
                "video_id": video_id,
                "video_url": url,
                "video_parts": chunk_filename,
                "frame_rate": frame_rate_chunk,
            })
        # Release the OpenCV2 capture object
        cap.release()


chunker(video_urls)
# Create the final data frame
df = pd.DataFrame(video_data)
print(df)


In [ ]:
import pandas as pd
import cv2
import os
import base64
import re
# List of YouTube video URLs
video_urls = [
    "https://www.youtube.com/watch?v=JDwduqfuHjc"
]

# Creating empty list to store the data for each video
video_data = []


# function to convert image to byte string
def image_to_byte_string(image):
    _, buffer = cv2.imencode('.jpg', image)
    byte_string = base64.b64encode(buffer).decode()
    return byte_string

# function to convert video to image


def vid2frames(video_chunk_num):
    # path to input video
    part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
    part_frame_folder_path = re.search(r"(p\d+)", video_chunk_num).group()
    part_frame_path = os.path.join(part_fpath, part_frame_folder_path)
    main_video_path = f'{os.path.join(part_fpath, video_chunk_num)}'

    # chunk_parts_video_fir_path p1,p2,p3,p4,p5
    print(part_frame_path)
    os.makedirs(part_frame_path, exist_ok=True)

    video_path = main_video_path
    # main_video_path VIDEid.mp4
    print(video_path)

    # capture video
    cap = cv2.VideoCapture(video_path)
    print(cap)
    # loop through frames and save as images
    frame_count = 1
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame_name = f'{part_frame_path}/{video_chunk_num}-frame{frame_count}.jpg'
            # save frame as image
            cv2.imwrite(frame_name, frame)
            print(f'{video_chunk_num}/frame{frame_count}.jpg')
            frame_count += 1
        else:
            break

    # release video capture and destroy window
    cap.release()
    cv2.destroyAllWindows()


def chunker(video_urls_list):
    # Loop through each video URL
    for url in video_urls_list:
        # Get the video ID from the URL
        video_id = url.split("=")[-1]
        print(f"NAT {video_id},,{url}")

        # Download the video using youtube-dl
        #os.system(f"youtube-dl -f 'bestvideo[ext=mp4]' -o 'video.mp4' {url}")

        temp_path = os.path.join(os.getcwd(), "videos")
        master_path = os.path.join(temp_path, video_id+".mp4")

        # Open the video using OpenCV2
        cap = cv2.VideoCapture(master_path)

        # Get the total number of frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Get the frame rate
        frame_rate = int(cap.get(cv2.CAP_PROP_FPS))

        # Split the video into 10-second chunks with two frames per second
        chunk_size = 10 * frame_rate
        for i in range(0, total_frames, chunk_size):
            # Set the start and end frames for the chunk
            start_frame = i
            end_frame = min(i + chunk_size, total_frames)
            # Get the chunk of frames
            frames = []
            for j in range(start_frame, end_frame):
                # Read the frame
                ret, frame = cap.read()
                # Keep only the first and middle frame
                if j == start_frame or j == (start_frame + end_frame) // 2:
                    frames.append(frame)

            # Save the chunk as an mp4 file
            chunk_filename = f"{video_id}-p{int(i/chunk_size)+1}.mp4"

            # path to input video
            part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
            os.makedirs(part_fpath, exist_ok=True)
            main_video_path = f'{os.path.join(part_fpath, chunk_filename)}'

            frame_rate_chunk = 2
            out = cv2.VideoWriter(f"{main_video_path}", cv2.VideoWriter_fourcc(
                *'mp4v'), frame_rate_chunk, (frames[0].shape[1], frames[0].shape[0]))
            for frame in frames:
                out.write(frame)
            out.release()

            # Add the video data to the list
            video_data.append({
                "video_id": video_id,
                "video_url": url,
                "video_parts": chunk_filename,
                "frame_rate": frame_rate_chunk,
            })
        # Release the OpenCV2 capture object
        cap.release()


def img2next(df_links):
    framxrate = 2
    df = df_links
    # add columns for each frame

    part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'

    for i in range(framxrate):
        df[f'frame_{i+1}'] = ''

    print("NANO TECG")

    # iterate over each row and extract frames
    for index, row in df.iterrows():
        video_chunk_num = row['video_parts']

        print(f"frame taet XDX {framxrate}")
        frames = []

        print(index)
        for i in range(int(framxrate)):
            part_frame_folder_path = re.search(
                r"(p\d+)", video_chunk_num).group()
            part_frame_path = os.path.join(part_fpath, part_frame_folder_path)
            main_video_path = f'{os.path.join(part_fpath, video_chunk_num)}'

            frame_path = f'{part_frame_path}\{video_chunk_num}-frame{i+1}.jpg'
            #frame_path = f'{video_parts[:-4]}-frame{i+1}.jpg'
            #frame_path = f'/home/siddharthsk7t/DL/DL-temp/frames/frame{i+1}.jpg'
            print(f"frame path {frame_path}")
            frame = cv2.imread(frame_path)
            byte_string = image_to_byte_string(frame)
            frames.append(byte_string)
            #print(f"byte string: {byte_string}")
            #print(f"fraem? string: {frames}")
            # os.remove(frame_path)

            print(frames)
        df.loc[index, df.columns[4:]] = frames
    return df


chunker(video_urls)
# Create the final data frame
df = pd.DataFrame(video_data)
for idx in df["video_parts"]:
    print(idx)
    vid2frames(idx)

df = img2next(df)


print(df)
